# <div align="center" style="color: #ff5733;">CIC Summary Data</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
sq = """with b1 as 
(SELECT
  customerId,
  digitalLoanAccountId,
  run_date,
  row_number() over(partition by customerId order by run_date desc) rnkmax,
  row_number() over(partition by customerId order by run_date) rnkmin,
  case when responseMsg like '%MORE SUBJECTS%' then 1 else 0 end responsemsg,
  case when DescisionValue like '%Approve%' then 1 else 0 end Decision_Approve,
  case when DescisionValue like '%Reject%' then 1 else 0 end Decision_Reject,
  case when DescisionValue like '%Review%' then 1 else 0 end Decision_Review,
  case when coalesce(DescisionValue, 'NA') like 'NA' then 1 else 0 end Decision_NA,
  case when coalesce(ExclusionRuleCode, 'NA') like 'NA' then digitalLoanAccountId end No_Exclusion,
  case when ExclusionRuleCode like '%ExG%' then digitalLoanAccountId end Noupdateinlast9months,
  case when ExclusionRuleCode like '%ExD%' then digitalLoanAccountId end contractsonlyasGuarantor,
  case when ExclusionRuleCode like '%ExE%' then digitalLoanAccountId end Openedcontractrecently,
  case when ExclusionRuleCode like '%ExC%' then digitalLoanAccountId end morethan50contracts,
  case when ExclusionRuleCode like '%ExI%' then digitalLoanAccountId end inactivelast18months,
  case when ExclusionRuleCode like '%ExH%' then digitalLoanAccountId end nominimumscoringcriteria,
  cast(coalesce(ScoreRaw, '-1') as int64) scoreraw,
  case when ScoreRange like '%Di%' then digitalLoanAccountId end ScoreRange_Di,
  case when ScoreRange like '%Bi%' then digitalLoanAccountId end ScoreRange_Bi,
  case when ScoreRange like '%Ci%' then digitalLoanAccountId end ScoreRange_Ci,
  case when ScoreRange like '%Hi%' then digitalLoanAccountId end ScoreRange_Hi,
  case when ScoreRange like '%Gi%' then digitalLoanAccountId end ScoreRange_Gi,
  case when ScoreRange like '%Ii%' then digitalLoanAccountId end ScoreRange_Ii,
  case when ScoreRange like '%Fi%' then digitalLoanAccountId end ScoreRange_Fi,
  case when ScoreRange like '%Ji%' then digitalLoanAccountId end ScoreRange_Ji,
  case when ScoreRange like '%Ei%' then digitalLoanAccountId end ScoreRange_Ei,
  case when ScoreRange is null then digitalLoanAccountId end ScoreRange_NoScore,
  case when cast(Accounts30Days as int64) + cast(Accounts30DaysOther as int64) > 0 then digitalLoanAccountId end Accounts30Daysflag,
  case when cast(Accounts60Days as int64) + cast(Accounts60DaysOther as int64) > 0 then digitalLoanAccountId end Accounts60Daysflag,
  case when cast(Accounts90Days as int64) + cast(Accounts90DaysOther as int64) > 0 then digitalLoanAccountId end Accounts90Daysflag,
  Accounts30Days,
  Accounts30DaysOther,
  Accounts60Days,
  Accounts60DaysOther,
  Accounts90Days,
  Accounts90DaysOther,
  AgeAtLoanTermination, 
  AgeAtLoanTerminationMax,
  CreditAvgCreditLimit,
  CreditHighestCreditLimit,
  CreditMaxOutstandingBalance,
  CreditMaxOverdueAmount,
CreditMaxOverdueDays,
CreditNumberOfContracts,
CreditTotalCreditLimit,
CreditTotalOutstandingBalanceAmount,
CreditTotalOverdueAmount,
InstMaxOverdueAmount,
InstMaxOverdueAmountDate,
InstMaxOverdueDays,
InstMaxOverdueDaysDate,
InstNumberOfContracts,
InstTotalMonthlyPaymentsAmount,
InstTotalOutstandingBalanceAmount,
InstTotalOverdueAmount,
LoanToValue,
MonthsOfBooks,
MonthsOfBooksOther,
NonInstAvgCreditLimit,
NonInstHighestCreditLimit,
NonInstNumberOfContracts,
NonInstTotalCreditLimit,
NonInstTotalOverdraftAmount,
NonInstTotalUtilization,
NumberOfContract,
NumberOfContractAC,
NumberOfContractACOther,
NumberOfContractCLCA,
NumberOfContractCLCAOther,
NumberOfContractOther,
NumberOfContractRF,
NumberOfContractRFOther,
NumberOfContractRN,
NumberOfContractRNOther,
NumberOfContractRQ,
NumberOfContractRQOther,
OverlimitFlag,
OverlimitFlagOther,
ReportingProvidersNumber,
ReportingProvidersNumberOther,
SubjectInfoTypeCode,
SubjectInfoTypeCodeDesc,
TotalOutstanding,
TotalOutstandingOther,
scoreLabelDesc,
scoreLabelValue,
FROM
  prj-prod-dataplatform.risk_credit_cic_data.cic_summary
  where date(run_date) between '2023-01-01' and '2023-10-31'
order by customerId, run_date, digitalLoanAccountId
)
select 
customerId, 
count(distinct digitalLoanAccountId) cnttonikloanid,
max(run_date) lastest_enquirydate,
min(run_date) Earliest_enquirydate,
count(distinct run_date) no_of_enquiries,
sum(case when rnkmax = 1 then Decision_Approve else 0 end) Decision_Approve,
sum(case when rnkmax = 1 then Decision_Reject else 0 end) Decision_Reject,
sum(case when rnkmax = 1 then Decision_Review else 0 end) Decision_Review,
sum(case when rnkmax = 1 then  Decision_NA else 0 end) Decision_NA,
max(case when responsemsg = 1 then 1 else 0 end) responsemsg_flag,
count(distinct case when rnkmax = 1 then No_Exclusion end) No_Exclusion,
count(distinct case when rnkmax = 1 then Noupdateinlast9months end) Noupdateinlast9months,
count(distinct case when rnkmax = 1 then contractsonlyasGuarantor end) contractsonlyasGuarantor,
count(distinct case when rnkmax = 1 then Openedcontractrecently end) Openedcontractrecently,
count(distinct case when rnkmax = 1 then morethan50contracts end) morethan50contracts,
count(distinct case when rnkmax = 1 then inactivelast18months end) inactivelast18months,
count(distinct case when rnkmax = 1 then nominimumscoringcriteria end) nominimumscoringcriteria,
max(scoreraw) max_scoreraw,
min(scoreraw) min_scoreraw,
max(case when rnkmax = 1 then scoreraw end) latestenqmaxscore,
max(case when rnkmin != rnkmax and rnkmin =1 then scoreraw end) earliestenqmaxsore, 
min(case when rnkmax = 1 then scoreraw end) latestenqminscore,
min(case when rnkmin != rnkmax and rnkmin =1 then scoreraw end) earliestenqminsore,
count(distinct case when rnkmax =1 then ScoreRange_Di end) ScoreRange_Di,
count(distinct case when rnkmax = 1 then ScoreRange_Bi end) ScoreRange_Bi,
count(distinct case when rnkmax =1 then ScoreRange_Ci end) ScoreRange_Ci,
count(distinct case when rnkmax = 1 then ScoreRange_Hi end) ScoreRange_Hi,
count(distinct case when rnkmax  = 1 then ScoreRange_Gi end) ScoreRange_Gi,
count(distinct case when rnkmax = 1 then ScoreRange_Ii end) ScoreRange_Ii,
count(distinct case when rnkmax = 1 then ScoreRange_Fi end) ScoreRange_Fi,
count(distinct case when rnkmax = 1 then ScoreRange_Ji end) ScoreRange_Ji,
count(distinct case when rnkmax = 1 then ScoreRange_Ei end) ScoreRange_Ei,
count(distinct case when rnkmax = 1 then ScoreRange_NoScore end) ScoreRange_NoScore,
max(case when rnkmax = 1 and Accounts30Daysflag is not null then 1 else 0 end) Accounts30Daysflag, 
max(case when rnkmax = 1 and Accounts60Daysflag is not null then 1 else 0 end) Accounts60Daysflag,
max(case when rnkmax = 1 and Accounts90Daysflag is not null then 1 else 0 end) Accounts90Daysflag,
max(case when rnkmax = 1 then Accounts30Days end) latesteq_Accounts30Days,
max(Accounts30Days) Accounts30Days_ever,
max(Accounts30DaysOther) Accounts30DaysOther_ever,
max(Accounts60Days) Accounts60Days_ever,
max(Accounts60DaysOther) Accounts60DaysOther_ever,
max(Accounts90Days) Accounts90Days_ever,
max(Accounts90DaysOther) Accounts90DaysOther_ever,
min(AgeAtLoanTermination) AgeAtLoanTermination_min,
max(AgeAtLoanTermination) AgeAtLoanTermination_max,
min(AgeAtLoanTerminationMax) AgeAtLoanTerminationMax_min,
max(AgeAtLoanTerminationMax) AgeAtLoanTerminationMax_max,
min(CreditAvgCreditLimit) CreditAvgCreditLimit_min,
max(CreditAvgCreditLimit) CreditAvgCreditLimit_max,
min(CreditHighestCreditLimit) CreditHighestCreditLimit_min,
max(CreditHighestCreditLimit) CreditHighestCreditLimit_max,
max(case when cast(CreditHighestCreditLimit as int64) = 0 then 0 else round(cast(CreditMaxOutstandingBalance as int64)/cast(CreditHighestCreditLimit as int64), 2) end) max_credit_untilization,
min(cast(CreditMaxOverdueAmount as int64)) CreditMaxOverdueAmount_min,
max(cast(CreditMaxOverdueAmount as int64)) CreditMaxOverdueAmount_max,
min(cast(case when CreditMaxOverdueDays = '' then '0' else coalesce(CreditMaxOverdueDays, '0') end as int64)) CreditMaxOverdueDays_min,
max(cast(case when CreditMaxOverdueDays = '' then '0' else coalesce(CreditMaxOverdueDays, '0') end as int64)) CreditMaxOverdueDays_max,
min(cast(CreditNumberOfContracts as int64)) CreditNumberOfContracts_min,
max(cast(CreditNumberOfContracts as int64)) CreditNumberOfContracts_max,
min(cast(CreditTotalCreditLimit as int64)) CreditTotalCreditLimit_min,
max(cast(CreditTotalCreditLimit as int64)) CreditTotalCreditLimit_max,
min(cast(CreditTotalOutstandingBalanceAmount as int64)) CreditTotalOutstandingBalanceAmount_min,
max(cast(CreditTotalOutstandingBalanceAmount as int64)) CreditTotalOutstandingBalanceAmount_max,
min(cast(CreditTotalOverdueAmount as int64)) CreditTotalOverdueAmount_min,
max(cast(CreditTotalOverdueAmount as int64)) CreditTotalOverdueAmount_max,
min(cast(InstMaxOverdueAmount as int64)) InstMaxOverdueAmount_min,
max(cast(InstMaxOverdueAmount as int64)) InstMaxOverdueAmount_max,
min(cast(case when InstMaxOverdueDays = '' then '0' else coalesce(InstMaxOverdueDays, '0') end as int64)) InstMaxOverdueDays_min,
max(cast(case when InstMaxOverdueDays = '' then '0' else coalesce(InstMaxOverdueDays, '0') end as int64)) InstMaxOverdueDays_max,
sum(case when rnkmax = 1 then cast(InstNumberOfContracts as int64) else 0 end) latest_enq_InstNumberOfContracts,
min(cast(InstTotalMonthlyPaymentsAmount as int64)) InstTotalMonthlyPaymentsAmount_min,
max(cast(InstTotalMonthlyPaymentsAmount as int64)) InstTotalMonthlyPaymentsAmount_max,
min(cast(InstTotalOutstandingBalanceAmount as int64)) InstTotalOutstandingBalanceAmount_min,
max(cast(InstTotalOutstandingBalanceAmount as int64)) InstTotalOutstandingBalanceAmount_max,
min(cast(InstTotalOverdueAmount as int64)) InstTotalOverdueAmount_min,
max(cast(InstTotalOverdueAmount as int64)) InstTotalOverdueAmount_max,
max(MonthsOfBooks) MonthsOfBooks_max,
max(MonthsOfBooksOther) MonthsOfBooksOther_max,
min(cast(NonInstAvgCreditLimit as int64)) NonInstAvgCreditLimit_min,
max(cast(NonInstAvgCreditLimit as int64)) NonInstAvgCreditLimit_max,
min(cast(NonInstHighestCreditLimit as int64)) NonInstHighestCreditLimit_min,
max(cast(NonInstHighestCreditLimit as int64)) NonInstHighestCreditLimit_max,
min(cast(NonInstNumberOfContracts as int64)) NonInstNumberOfContracts_min,
max(cast(NonInstNumberOfContracts as int64)) NonInstNumberOfContracts_max,
min(cast(NonInstTotalOverdraftAmount as int64)) NonInstTotalOverdraftAmount_min,
max(cast(NonInstTotalOverdraftAmount as int64)) NonInstTotalOverdraftAmount_max,
min(cast(NonInstTotalUtilization as int64)) NonInstTotalUtilization_min,
max(cast(NonInstTotalUtilization as int64)) NonInstTotalUtilization_max,
max(case when rnkmax = 1 then cast(NumberOfContract as int64) else 0 end) latest_enq_NumberOfContract,
max(case when rnkmax = 1 then cast(NumberOfContractOther as int64) else 0 end) latest_enq_NumberOfContractOther,
max(case when rnkmax = 1 then cast(NumberOfContractAC as int64) else 0 end) latest_enq_NumberOfContractAC,
max(case when rnkmax = 1 then cast(NumberOfContractACOther as int64) else 0 end) latest_enq_NumberOfContractACOther,
max(case when rnkmax = 1 then cast(NumberOfContractCLCA as int64)else 0 end) latest_enq_NumberOfContractCLCA,
max(case when rnkmax = 1 then cast(NumberOfContractCLCAOther as int64) else 0 end) latest_enq_NumberOfContractCLCAOther,
max(case when rnkmax = 1 then cast(NumberOfContractRF as int64) else 0 end) latest_enq_NumberOfContractRF,
max(case when rnkmax = 1 then cast(NumberOfContractRFOther as int64) else 0 end) latest_enq_NumberOfContractRFOther,
max(case when rnkmax = 1 then cast(NumberOfContractRN as int64) else 0 end) latest_enq_NumberOfContractRN,
max(case when rnkmax = 1 then cast(NumberOfContractRNOther as int64) else 0 end) latest_enq_NumberOfContractRNOther,
max(case when rnkmax = 1 then cast(NumberOfContractRQ as int64) else 0 end) latest_enq_NumberOfContractRQ,
max(case when rnkmax = 1 then cast(NumberOfContractRQOther as int64) else 0 end) latest_enq_NumberOfContractRQOther,
max(case when coalesce(OverlimitFlag, 'NA') like 'true' then 1 else 0 end) OverlimitFlag,
max(case when coalesce(OverlimitFlagOther, 'NA') like 'true' then 1 else 0 end) OverlimitFlagOther ,
max(case when rnkmax = 1 then cast(TotalOutstanding as int64) else 0 end) Latest_enq_TotalOutstanding,
max(case when rnkmax = 1 then cast(TotalOutstandingOther as int64) else 0 end) Latest_enq_TotalOutstandingOther,
max(case when rnkmax = 1 then scoreLabelDesc end) Latest_enq_scoreLabelDesc
from b1
group by 1"""

In [3]:
cicsummarydf = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of cicsummarydf is:\t {cicsummarydf.shape }")

Job ID 5045e63c-e868-420a-81fb-0c9039064e7e successfully executed: |          |
Downloading: 100%|██████████|
The shape of cicsummarydf is:	 (151379, 104)


In [4]:
cicsummarydf.to_csv("cicsummarydf_20240517.csv", index = False)